In [ ]:
from pathlib import Path

seeds = range(1, 11)

# Config should match Problem init

base_path = Path("smacbenchmarking/configs/problem")

# Problem ids
benchmark_id_bbob = "BBOB"

# BBOB

In [ ]:
# BBOB
from omegaconf import DictConfig, OmegaConf

import shutil

target_path = base_path / benchmark_id_bbob
if target_path.is_dir():
    shutil.rmtree(str(target_path))
target_path.mkdir(parents=True, exist_ok=True)

problem_class = "smacbenchmarking.benchmarks.bbob.BBOBProblem"

fids = list(range(1, 25))
instances = [0]
dimensions = [4]
n_trials = {
    4: 100,
}

for fid in fids:
    for inst in instances:
        for dim in dimensions:
            problem_id = f"noiseless/{dim}/{fid}/{inst}"
            cfg = DictConfig({
                "benchmark_id": benchmark_id_bbob,
                "problem_id": problem_id,
                "problem": {
                    # "problem_class": {
                        "_target_": problem_class,
                        "dimension": dim,
                        "fid": fid,
                        "instance": inst,
                        "seed": "${seed}",  # get the seed from global cfg                      
                    # },
                },
                "task": {
                    "n_trials": n_trials[dim],
                    "n_objectives": 1,
                }
            
            })
            fn = target_path / f"cfg_{dim}_{fid}_{dim}_{inst}.yaml"
            yaml_str = OmegaConf.to_yaml(cfg=cfg)
            yaml_str = "# @package _global_\n" + yaml_str
            fn.write_text(yaml_str)
            print(cfg)



# YAHPO